In [1]:
# Import path configuration
from pathlib import Path
import sys
import logging
from dotenv import load_dotenv

from langchain_core.messages import (
    AIMessage,
    AnyMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
load_dotenv()
logger  = logging.getLogger(__name__)

def _find_project_root(start: Path) -> Path:
    for parent in [start, *start.parents]:
        if (parent/"pyproject.toml").exists():
            return parent
    raise RuntimeError("Project root not found")

ROOT = _find_project_root(Path.cwd())
SRC = ROOT / "src"

if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC)) 

In [2]:

from app.utils.test import print_something
from app.adapters.langgraph_agent import LangGraphAgent, MODEL

In [3]:
print_something("This is a message im Englische")

Dis ist ein Nachricht This is a message im Englische


In [ ]:

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver

tool = TavilySearchResults(max_results=2)
# memory = SqliteSaver.from_conn_string(":memory:")
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatGoogleGenerativeAI(model=MODEL)
messages = [HumanMessage(content="What is the weather in sf?")]
thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = LangGraphAgent(model, [tool], system=prompt, checkpointer=memory)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            logger.info(v["messages"])
            print(v["messages"])

/tmp/ipykernel_211466/3491619815.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)
